# __The Basics of Object Oriented Programming__

<br>

__Finance 5350: Computational Financial Modeling__ <br>
__Tyler J. Brough__ <br>
__November 4, 2018__






## __The Basics of Object Oriented Programming__


Object Oriented Programming (OOP) is based on a few foundation concepts, namely:

1. Polymorphism
2. Encapsulation
3. Inheritance

    Often a fourth concept is added (one that we will use):

4. Composition


### __Polymorphism__ 

Let's start with polymorphism.

See here: https://en.wikipedia.org/wiki/Polymorphism_(computer_science)

The etymology of the word suggests a meaning similar to _"taking many shapes."_ What this means for programming is that we set up some basic interfaces and then establish different types that instantiate that interface. In OOP we seek to represent real world entities or ___objects___ as software types. Then we can operate on these types in meaningful ways. Let's see this with and example that is meaningful to us as financial economists and financial engineers: the real world example of an option contract. We can create a new type with the keyword `class` as follows:

In [1]:
import numpy as np
from scipy.stats import binom

In [2]:
class Option:
    def __init__(self, expiry, strike):
        self.expiry = expiry
        self.strike = strike
        
    def payoff(self):
        raise NotImplementedError("Inheriting classes must implement this method!")

So far this class is not that useful, but we will make it more useful in short order. For now let's see how we can access the data contained in the class (called class attributes). 

In [3]:
theOption = Option(1.0, 40.0)

In [4]:
theOption.expiry

1.0

In [5]:
theOption.strike

40.0

In [6]:
theOption.payoff()

NotImplementedError: Inheriting classes must implement this method!

### Inheritance

Let's see how we can use the concept of inheritance to make our class more useful for polymorphism. 

In [7]:
class CallOption(Option):
    def payoff(self, spot):
        return np.maximum(spot - self.strike, 0.0)
        
class PutOption(Option):
    def payoff(self, spot):
        return np.maximum(self.strike - spot, 0.0)

In [8]:
spot = 41.0
theCall = CallOption(1.0, 40.0)
theCall.payoff(spot)

1.0

In [9]:
spot = 39.0
thePut = PutOption(1.0, 40.0)
thePut.payoff(spot)

1.0

Now let's return to our `binomialPricer` function to see how we can benefit from polymorphism with these new types.

In [10]:
def binomialPricer(option, S, r, v, q, n):
    nodes = n  + 1
    T = option.expiry
    K = option.strike
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pstar = (np.exp((r - q) * h) - d) / (u - d)
    
    price = 0.0
    
    for i in range(nodes):
        prob = binom.pmf(i, n, pstar)
        spotT = S * (u ** i) * (d ** (n - i))
        po = option.payoff(spotT) 
        price += po * prob
        
    price *= np.exp(-r * T)
    
    return price

In [11]:
spot = 41.0
rate = 0.08
vol = 0.30
expiry = 1.0
strike = 40.0
div = 0.0
nsteps = 500

theCall = CallOption(expiry, strike)
theCallPrc = binomialPricer(theCall, spot, rate, vol, div, nsteps)
print(f"The Call Option Price is: {theCallPrc : 0.4f}")

The Call Option Price is:  6.9603


BUT we can use this same exact function without changing a thing about it now to price a put option as well!

In [12]:
thePut = PutOption(expiry, strike)
thePutPrc = binomialPricer(thePut, spot, rate, vol, div, nsteps)
print(f"The Put Price is: {thePutPrc : 0.4f}")

The Put Price is:  2.8849


This is polymorphism at work:

- The function `binomialPricer` takes an `Option` type. 
- With inheritance we can make sub-types. 
- Anywhere a parent type can go the child type can go. 
- Thus a `CallOption` is an `Option` and a `PutOption` is an `Option`. 
- So the class `Option` provides an interface through which we can send different sub-types to the algorithm.

We only have to write this algorithm one time and it can take different kinds of `Option` sub-types that bring with them their own `payoff` methods, and thus their own behavior. This is the meaning of polymorphism! And we can see how the concept of inheritance makes it possible.

Notice that we could ship to clients and they could create their own sub-types that would work with our algorithm. That is an amazing thing!

### Encapsulation

The princple of encapsulation is a sort of best practice that says that we should hide data from the client that is only needed internally for the algorithm. When you think _"encapsulation"_ think the phrase _"data hiding."_ We have actually already seen this:

In [13]:
thePut.expiry

1.0

In [14]:
thePut.strike

40.0

We have hidden these variables behind the class infrastructure. But as we can see they are readily available. In `Python` the user can change them using the dot notation ("."). 

In [15]:
thePut.expiry = 0.75

In [16]:
thePut.expiry

0.75

This is not strict encapsulation, which would say that the end user should not be allowed to change these values that are operated on interally within the class. Python allows us to handle this though with a slight modification to our base class:

In [17]:
class Option:
    def __init__(self, expiry, strike):
        self.__expiry = expiry
        self.__strike = strike
        
    def payoff(self):
        raise NotImplementedError("Inheriting classes must implement this method!")

The "dunder" i.e. the `__` makes it so that the end user cannot change these values willy nilly. Let's re-implement the subclasses and see how this works:

In [18]:
class CallOption(Option):
    def payoff(self, spot):
        return np.maximum(spot - self.__strike, 0.0)
        
class PutOption(Option):
    def payoff(self, spot):
        return np.maximum(self.__strike - spot, 0.0)

In [19]:
theCall = CallOption(1.0, 40.0)

In [20]:
theCall.strike

AttributeError: 'CallOption' object has no attribute 'strike'

In [21]:
theCall.__strike

AttributeError: 'CallOption' object has no attribute '__strike'

With this extra step we can enforce a stricter encapsulation or "data hiding." This allows us to provide a clean interface to clients while guaranteeing that they won't be able to mess anything up interally by writing code externally.

<br>

It could be true, however that we want to allow end users to be able to change these values for their purposes. We can allow this but by providing an interface to do this while still achieving encapsulation with what are called `Properties`. Let's see how we can provide a property to allow access to class attributes. 

In [22]:
class Option:
    def __init__(self, expiry, strike):
        self.__expiry = expiry
        self.__strike = strike
        
    def payoff(self):
        raise NotImplementedError("Inheriting classes must implement this method!")

    @property
    def expiry(self):
        return self.__expiry
    
    @property
    def strike(self):
        return self.__strike

class CallOption(Option):
    def payoff(self, spot):
        return np.maximum(spot - self.__strike, 0.0)
       
class PutOption(Option):
    def payoff(self, spot):
        return np.maximum(self.__strike - spot, 0.0)

In [23]:
theCall = CallOption(1.0, 40.0)
theCall.expiry

1.0

In [24]:
theCall.strike

40.0

But we still won't allow the end user to set the values to something new.

In [25]:
theCall.expiry = 0.75

AttributeError: can't set attribute

We can allow this with a setter property:

In [26]:
class Option:
    def __init__(self, expiry, strike):
        self.__expiry = expiry
        self.__strike = strike
        
    def payoff(self):
        raise NotImplementedError("Inheriting classes must implement this method!")

    @property
    def expiry(self):
        return self.__expiry
    
    @property
    def strike(self):
        return self.__strike
    
    @expiry.setter
    def expiry(self, newExpiry):
        self.__expiry = newExpiry
        
    @strike.setter
    def strike(self, newStrike):
        self.__strike = newStrike

class CallOption(Option):
    def payoff(self, spot):
        return np.maximum(spot - self.__strike, 0.0)
       
class PutOption(Option):
    def payoff(self, spot):
        return np.maximum(self.__strike - spot, 0.0)

In [27]:
theCall = CallOption(1.0, 40.0)

In [28]:
theCall.expiry

1.0

In [29]:
theCall.expiry = 0.75
theCall.expiry

0.75

This is still principled encapsulation because, while we are allowing the getting and setting of the data attributes we are guaranteeing that the end user does so only through the sanctioned interface that we provide. Thus we can allow flexibility while still keep our code base clean and safe from getting messed up from external use. 

<br>

So, in review, we've seen the basics of OOP: 

1. Polymorphism
2. Inheritance
3. Data encapsulation. 

Let's now talk about composition. 



### __Composition__

When you think about inheritance you want to think about an _"is a"_ relationship. For example, a `CallOption` is an `Option`. And a `PutOption` is an `Option`. You can build up pretty deep hierarchies using inheritance. 

<br>

An alternative is to use the concept of composition. When you think about composition you want to think about a _"has a"_ relationship. For example, we could have made it so that an `Option` has a `payoff` function. This probably matches the real-world scenario a little bit better. There is a big war over whether one should use inheritance or composition to create objects. 

<br>

Practically speaking, we can use both to do what we want. Let's see how we can use the best of both:

In [30]:
class VanillaPayoff:
    def __init__(self, strike):
        self.strike = strike

    def value(self, spot):
        raise NotImplementedError("Subclasses must implement this method!")
                
class VanillaCallPayoff(VanillaPayoff):
    def value(self, spot):
        return np.maximum(spot - self.strike, 0.0)
       
class VanillaPutPayoff(VanillaPayoff):
    def value(self, spot):
        return np.maximum(self.strike - spot, 0.0)

class Option:
    def __init__(self, expiry, payoff):
        self.__expiry = expiry
        self.__payoff = payoff
        
    def payoff(self, spot):
        return self.__payoff.value(spot)

    @property
    def expiry(self):
        return self.__expiry
    
    @expiry.setter
    def expiry(self, newExpiry):
        self.__expiry = newExpiry

In [31]:
callPO = VanillaCallPayoff(40.0)
callPO.strike

40.0

In [32]:
callPO.value(41.0)

1.0

In [33]:
theCall = Option(1.0, callPO)
theCall.payoff(41.0)

1.0

This works, but is a bit cumbersome. Since functions are first class objects in `Python` we don't really need to use inheritance for the payoffs. Functions will do just fine. 

In [34]:
def callPayoff(spot, strike):
        return np.maximum(spot - strike, 0.0)
       
def putPayoff(spot, strike):
        return np.maximum(strike - spot, 0.0)

class Option:
    def __init__(self, expiry, strike, payoff):
        self.__expiry = expiry
        self.__strike = strike
        self.__payoff = payoff
        
    @property
    def expiry(self):
        return self.__expiry
    
    @expiry.setter
    def expiry(self, newExpiry):
        self.__expiry = newExpiry
        
    @property
    def strike(self):
        return self.__strike
    
    @strike.setter
    def strike(self, newStrike):
        self.__strike = newStrike
        
    def payoff(self, spot):
        return self.__payoff(spot, self.__strike)        

In [35]:
theCall = Option(1.0, 40.0, callPayoff)
theCall.payoff(41.0)

1.0

In [36]:
thePut = Option(1.0, 40.0, putPayoff)
thePut.payoff(39.0)

1.0

In [37]:
binomialPricer(thePut, spot, rate, vol, div, nsteps)

2.8849317768040366

In [38]:
binomialPricer(theCall, spot, rate, vol, div, nsteps)

6.9602779213377195